# Feagin's Order 10, 12, and 14 Methods

DifferentialEquations.jl includes Feagin's explicit Runge-Kutta methods of orders 10/8, 12/10, and 14/12. These methods have such high order that it's pretty much required that one uses numbers with more precision than Float64. As a prerequisite reference on how to use arbitrary number systems (including higher precision) in the numerical solvers, please see the Solving Equations in With Chosen Number Types notebook.

## Investigation of the Method's Error

We can use Feagin's order 16 method as follows. Let's use the twoDimlinearODEExample from previous examples. Like in the Solving Equations in With Chosen Number Types notebook, we change the initial condition to BigFloats to tell the solver to use BigFloat types.

In [1]:
using DifferentialEquations
prob = twoDimlinearODEExample(α=ones(BigFloat,4,2),u₀=map(BigFloat,rand(4,2)).*ones(4,2)/2)
sol =solve(prob::ODEProblem,Δt=1//16,alg=:Feagin14,adaptive=false);

Let's print the errors and find out how accurate it is:

In [2]:
print(sol.errors)

Dict(:l∞=>1.834920846276128345768456848043647539688554668437028441297187579147565729397152e-23,:final=>8.325176120322795020599060812643279837932924795032522034303070091950178990420393e-24,:l2=>5.158396995918330946892812981239371274402680113660348929191427359885823789028183e-24)

Compare that to machine $\epsilon$ for Float64:

In [3]:
eps(Float64)

2.220446049250313e-16

The error for Feagin's method when the stepsize is 1/16 is 8 orders of magnitude below machine $\epsilon$! However, that is dependent on the stepsize. If we instead use adaptive timestepping with the default tolerances [and set the internalnorm=1 to avoid [this issue](https://github.com/JuliaLang/julia/issues/17728)], we get

In [4]:
sol =solve(prob::ODEProblem,Δt=Δts[1],alg=:Feagin14,adaptive=true,internalnorm=1); 
println(sol.errors); print("The length was $(length(sol))")

LoadError: LoadError: UndefVarError: Δts not defined
while loading In[4], in expression starting on line 1

Notice that when the stepsize is much higher, the error goes up quickly as well. These super high order methods are best when used to gain really accurate approximations (using still modest timesteps). Some examples of where such precision is necessary is astrodynamics where the many-body problem is highly chaotic and thus sensitive to small errors.

## Convergence Test

The Order 14 method is awesome, but we need to make sure it's really that awesome. The following convergence test is used in the package tests in order to make sure the implementation is correct. Note that all methods have such tests in place.

In [5]:
Δts = 1.//2.^(10:-1:4)
sim = test_convergence(Δts,prob,alg=:Feagin14)

DifferentialEquations.ConvergenceSimulation of length 7.
Convergence Estimates: (l∞,14.290261124780095) (final,14.290261124780095) (l2,14.29968233607688)

For a better view of what's going on, let's plot the simulation results.

In [ ]:
Plots.plotly()
plot(sim)

Note that the PyPlot backend does not show this correctly. However, it's the only Plots.jl backed whose picture saves in the notebook in a format which Github can render. Therefore I can use `Plots.plotly()` to change to the Plotly backend and save the plot. The saved plot looks like:

![FeaginConvergence](https://raw.githubusercontent.com/ChrisRackauckas/DifferentialEquations.jl/master/examples/plots/FeaginConvergence.png)

This is a clear trend indicating that the convergence is truely Order 14. 